In [2]:
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle
import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras import optimizers
from string import punctuation
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional

In [3]:
!git clone https://github.com/Jayagn/Fake-news-challenge.git

fatal: destination path 'Fake-news-challenge' already exists and is not an empty directory.


In [120]:
#Specifying the folder locations
GloVe_DIR = './data/glove.twitter.27B.50d.txt'
DATA_DIR = './data'
MAX_SENT_LEN = 150 #to be tested on (150, 300 and 700)
MAX_VOCAB_SIZE = 28000 #vocabulary
BATCH_SIZE = 512
N_EPOCHS = 20 
LSTM_DIM = 50
EMBEDDING_DIM = 50 

In [121]:
seed = 1
np.random.seed(seed)

In [122]:
# Read the text files of fnc data
bodies = pd.read_csv(DATA_DIR + '/body_id.csv')
train_df = pd.read_csv(DATA_DIR + '/train.csv')
#validation_df = pd.read_csv(DATA_DIR + '/validation_data.csv')
test_df = pd.read_csv(DATA_DIR + '/test.csv')

In [123]:
train_df.replace('unrelated',1,True)
train_df.replace('agree',2,True)
train_df.replace('disagree',3,True)
train_df.replace('discuss',4,True)
combine_df_train = train_df.join(bodies.set_index('Body ID'), on='Body ID')
combine_df_test = test_df.join(bodies.set_index('Body ID'), on='Body ID')

In [125]:
#Pre-processing the data
word_seq_head_train = [text_to_word_sequence(head) for head in combine_df_train['Headline']]
word_seq_bodies_train = [text_to_word_sequence(body) for body in combine_df_train['articleBody']]
word_seq_head_test = [text_to_word_sequence(head) for head in combine_df_test['Headline']]
word_seq_bodies_test = [text_to_word_sequence(body) for body in combine_df_test['articleBody']]

In [126]:
word_seq = []
for i in range(len(word_seq_head_train)):
    word_seq.append(word_seq_head_train[i])
for i in range(len(word_seq_bodies_train)):
    word_seq.append(word_seq_bodies_train[i])
for i in range(len(word_seq_head_test)):
    word_seq.append(word_seq_head_test[i])
for i in range(len(word_seq_bodies_test)):
    word_seq.append(word_seq_bodies_test[i])

In [ ]:
#Tokenizing sentences
filter_list = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters=filter_list)
tokenizer.fit_on_texts([seq for seq in word_seq])

In [128]:
#Combining headline and body together
word_seq_train = [list(i) for i in word_seq_head_train]
for i in range(len(word_seq_head_train)):
    word_seq_train[i].extend(word_seq_bodies_train[i]) 
    
word_seq_test = [list(i) for i in word_seq_head_test]
for i in range(len(word_seq_head_test)):
    word_seq_test[i].extend(word_seq_bodies_test[i])

In [129]:
#Padding the data
X_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_train])
X_train = pad_sequences(X_train, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
y_train = combine_df_train['Stance']

In [130]:
#Converting the sequence of words to sequnce of indices
X_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_test])
X_test = pad_sequences(X_test, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

In [131]:
#One hot encoding
encoder_train = LabelEncoder()
encoder_train.fit(y_train)
encoded_train = encoder_train.transform(y_train)
dummy_y_train = np_utils.to_categorical(encoded_train)

In [132]:
X_train, X_vali, y_train, y_vali = train_test_split(X_train, dummy_y_train, random_state=10, test_size=0.1)

In [133]:
#GloVes embedding
glove_input_file = GloVe_DIR
word2vec_output_file = 'glove.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
embeddings = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [134]:
#Create an embedding matrix containing only the word's in our vocabulary
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items(): 
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None

    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
        
del embeddings

# LSTM with Attention Layer

In [113]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [116]:
model_4 = Sequential()
model_4.add(Embedding(input_dim=len(tokenizer.word_index)+1, input_length = MAX_SENT_LEN,
                          output_dim=EMBEDDING_DIM,
                          weights = [embeddings_matrix], trainable= False, name='word_embedding_layer', 
                          mask_zero=True)) # trainable=True results in overfitting

model_4.add(LSTM(LSTM_DIM, return_sequences=True, name='lstm_layer')) # Can try Bidirectional-LSTM
model_4.add(Dropout(rate=0.2, name='dropout_1')) # Can try varying dropout rates, in paper suggest 0.2
model_4.add(Activation(activation='relu', name='activation_1'))
#output (batch_size, timesteps, input_dim)
model_4.add(attention())
model_4.add(Dense(4, activation='softmax', name='output_layer'))

In [117]:
model_4.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 700, 50)           1686400   
_________________________________________________________________
lstm_layer (LSTM)            (None, 700, 50)           20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 700, 50)           0         
_________________________________________________________________
activation_1 (Activation)    (None, 700, 50)           0         
_________________________________________________________________
attention_1 (attention)      (None, 50)                750       
_________________________________________________________________
output_layer (Dense)         (None, 4)                 204       
Total params: 1,707,554
Trainable params: 21,154
Non-trainable params: 1,686,400
______________________________________

In [118]:
optimizer = optimizers.Adam(lr=0.001) # Try a different learning rate

model_4.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [119]:
history_4 = model_4.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=20,
          validation_data=(X_vali, y_vali)
         )
model_4.save('lstm_with_attention_700.h5')

Epoch 1/20
71/71 [==============================] - 198s 3s/step - loss: 0.9084 - accuracy: 0.6916 - val_loss: 0.8012 - val_accuracy: 0.7299
Epoch 2/20
71/71 [==============================] - 197s 3s/step - loss: 0.7829 - accuracy: 0.7353 - val_loss: 0.7861 - val_accuracy: 0.7299
Epoch 3/20
71/71 [==============================] - 198s 3s/step - loss: 0.7676 - accuracy: 0.7355 - val_loss: 0.7731 - val_accuracy: 0.7306
Epoch 4/20
71/71 [==============================] - 198s 3s/step - loss: 0.7554 - accuracy: 0.7355 - val_loss: 0.7612 - val_accuracy: 0.7306
Epoch 5/20
71/71 [==============================] - 198s 3s/step - loss: 0.7434 - accuracy: 0.7363 - val_loss: 0.7571 - val_accuracy: 0.7326
Epoch 6/20
71/71 [==============================] - 199s 3s/step - loss: 0.7305 - accuracy: 0.7385 - val_loss: 0.7441 - val_accuracy: 0.7361
Epoch 7/20
71/71 [==============================] - 199s 3s/step - loss: 0.7423 - accuracy: 0.7376 - val_loss: 0.8017 - val_accuracy: 0.7323
Epoch 8/20
71